In [1]:
using CSV, Tables

In [2]:
path = "data/"

"data/"

In [3]:
train_data = CSV.File(string(path, "atoms-train.csv"); header=false, delim="\t", quotechar=' ') |> Tables.matrix;
test_data = CSV.File(string(path, "atoms-test.csv"); header=false, delim="\t", quotechar=' ') |> Tables.matrix;

In [4]:
train_data[6, 19]

"PROPN"

3 - pos  
4 - tag  
5 - dep  
7 - head pos  
8- head tag  
9 - head dep  
20 - pos in previous word

In [5]:
# redundant_cols = [2, 6, 15, 16, 26];
features = [3, 4, 5, 7, 8, 9, 20]
# features = [3, 4, 5, 9, 20]
# cols = [i for i in 2:26 if (i in redundant_cols)];

In [6]:
X_train = train_data[:, features]
y_train = train_data[:, 1]
X_test = test_data[:, features]
y_test = test_data[:, 1];

In [7]:
using Random
Random.seed!(123)

MersenneTwister(123)

In [8]:
using DataFrames

In [9]:
function one_hot(df, col, uniques)
    if length(uniques) == 0
        ux = unique(df[!, col])
    else
        ux = uniques
    end
    df = transform(df, @. "$col" => ByRow(isequal(ux)) .=> Symbol("$(col)_", ux))
    if length(uniques) == 0
        return df, ux
    else
        return df
    end
end

one_hot (generic function with 1 method)

In [10]:
train = DataFrame(X_train, :auto);
rename!(train, [1 => :pos, 2 => :tag, 3 => :dep, 4 => :head_pos, 
             5 => :head_tag, 6 => :head_dep, 7 => :pos_prev])
uniques = Dict()
for column in names(train)
    train, unique = one_hot(train, column, [])
    uniques[column] = unique
end
features = [i for i in range(8, stop=size(train)[2])];
train = train[!, features];

In [11]:
test = DataFrame(X_test, :auto);
rename!(test, [1 => :pos, 2 => :tag, 3 => :dep, 4 => :head_pos, 
             5 => :head_tag, 6 => :head_dep, 7 => :pos_prev]);

In [12]:
for column in names(test)
    test = one_hot(test, column, uniques[column])
end

In [13]:
test = test[!, features];

In [14]:
X_train = Matrix(train);
X_test = Matrix(test);

In [15]:
using DecisionTree

In [16]:
model = DecisionTreeClassifier()

DecisionTreeClassifier
max_depth:                -1
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  nothing
root:                     nothing

In [17]:
replace!(X_train, missing => "None")
replace!(X_test, missing => "None");

In [18]:
fit!(model, X_train, y_train)

DecisionTreeClassifier
max_depth:                -1
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  Any["B", "C", "J", "M", "P", "T", "X"]
root:                     Decision Tree
Leaves: 252
Depth:  30

In [19]:
using Statistics
accuracy(x, y, model) = mean(predict(model, x) .== y)
accuracy(X_test, y_test, model)

0.9457120377655389

In [20]:
accuracy(X_train, y_train, model)

0.9908966772872099

In [21]:
features = [3, 4, 5, 9, 20]

5-element Vector{Int64}:
  3
  4
  5
  9
 20

In [22]:
X_train = train_data[:, features]
X_test = test_data[:, features];

In [23]:
train = DataFrame(X_train, :auto);
rename!(train, [1 => :pos, 2 => :tag, 3 => :dep, 4 => :head_dep, 5 => :pos_prev])
uniques = Dict()
for column in names(train)
    train, unique = one_hot(train, column, [])
    uniques[column] = unique
end
features = [i for i in range(6, stop=size(train)[2])];
train = train[!, features];

In [24]:
test = DataFrame(X_test, :auto);
rename!(test, [1 => :pos, 2 => :tag, 3 => :dep, 4 => :head_dep, 5 => :pos_prev]);

In [25]:
for column in names(test)
    test = one_hot(test, column, uniques[column])
end

In [26]:
test = test[!, features];

In [27]:
X_train = Matrix(train);
X_test = Matrix(test);

In [33]:
model = DecisionTreeClassifier(max_depth=15)

DecisionTreeClassifier
max_depth:                15
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  nothing
root:                     nothing

In [34]:
replace!(X_train, missing => "None")
replace!(X_test, missing => "None");

In [35]:
fit!(model, X_train, y_train)

DecisionTreeClassifier
max_depth:                15
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  Any["B", "C", "J", "M", "P", "T", "X"]
root:                     Decision Tree
Leaves: 166
Depth:  15

In [36]:
using Statistics
accuracy(x, y, model) = mean(predict(model, x) .== y)
accuracy(X_test, y_test, model)

0.9461054287962234

In [37]:
accuracy(X_train, y_train, model)

0.9733727810650887

In [38]:
rf = RandomForestClassifier(n_trees=100)
fit!(rf, X_train, y_train)

RandomForestClassifier
n_trees:             100
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             Any["B", "C", "J", "M", "P", "T", "X"]
ensemble:            Ensemble of Decision Trees
Trees:      100
Avg Leaves: 353.41
Avg Depth:  36.83

In [39]:
accuracy(X_test, y_test, rf)

0.9492525570416994

In [40]:
accuracy(X_train, y_train, model)

0.9733727810650887

In [93]:
c = [75, 27, 2, 5, 73]
println(names(train)[c])

["dep_punct", "tag_IN", "pos_NOUN", "pos_VERB", "dep_cc"]


In [94]:
print_tree(model, 5)

Feature 75, Threshold true
L-> Feature 27, Threshold true
    L-> Feature 2, Threshold true
        L-> Feature 5, Threshold true
            L-> Feature 73, Threshold true
                L-> 
                R-> J : 141/141
            R-> Feature 24, Threshold true
                L-> 
                R-> M : 37/37
        R-> Feature 116, Threshold true
            L-> Feature 78, Threshold true
                L-> 
                R-> M : 3/3
            R-> Feature 150, Threshold true
                L-> C : 1/1
                R-> 
    R-> Feature 111, Threshold true
        L-> Feature 110, Threshold true
            L-> Feature 107, Threshold true
                L-> 
                R-> 
            R-> Feature 147, Threshold true
                L-> B : 24/24
                R-> B : 16/18
        R-> Feature 146, Threshold true
            L-> Feature 152, Threshold true
                L-> 
                R-> B : 11/13
            R-> B : 1/2
R-> Feature 108, Threshold tru

In [98]:
using BSON

In [102]:
BSON.@save "models/model_dt.bson" model uniques

In [103]:
BSON.@save "models/model_rf.bson" rf uniques